In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split

import torchvision.models as models
from torchvision import transforms
from torchvision.datasets import ImageFolder

import numpy as np
import pandas as pd
from PIL import Image

In [22]:
# define a custom dataset class
class FashionMNISTDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file) # read data from csv file as a pandas dataframe
        self.transform = transform # initial transfrom

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = Image.fromarray(self.data_frame.iloc[idx, 1:].values.reshape(28, 28).astype(np.uint8))
        
        label = int(self.data_frame.iloc[idx, 0])

        if self.transform:
            image = self.transform(image)

        return image, label

In [23]:
#Data Preprocessing
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.AutoAugment(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4850, 0.4560, 0.4060], std=[0.2290, 0.2240, 0.2250])
])

test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4850, 0.4560, 0.4060], std=[0.2290, 0.2240, 0.2250])
])

In [24]:
#define dataset and dataloader
train_path = 'dataset/fashion-mnist_train.csv'
test_path = 'dataset/fashion-mnist_test.csv'

train_data = FashionMNISTDataset(csv_file=train_path, transform=train_transform)
test_data = FashionMNISTDataset(csv_file=test_path, transform=test_transform)
testLen = int(len(test_data) * 0.5)
valLen = len(test_data) - testLen
test_data, val_data = random_split(test_data, [testLen, valLen])

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [25]:
#set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

#choose a model
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)

#choose a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), weight_decay=0.001, lr=0.0001)

#move model to cuda/cpu
model.to(device)

cuda


d:\miniconda3\envs\summercourse\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\miniconda3\envs\summercourse\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [26]:
#training section
num_epochs = 80
best_accuracy = 0.0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = correct_train / total_train

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

    #evaluate on validation set every epoch
    if epoch % 5 == 4:
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

            val_loss = running_val_loss / len(val_loader.dataset)
            val_accuracy = correct_val / total_val

            if val_accuracy > best_accuracy:
                # Save model weights
                torch.save(model.state_dict(), 'best_model.pth')
                best_accuracy = val_accuracy

            print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

Epoch 1/80, Training Loss: 0.6486, Training Accuracy: 0.7620
Epoch 2/80, Training Loss: 0.4644, Training Accuracy: 0.8257
Epoch 3/80, Training Loss: 0.4140, Training Accuracy: 0.8464
Epoch 4/80, Training Loss: 0.3767, Training Accuracy: 0.8593
Epoch 5/80, Training Loss: 0.3549, Training Accuracy: 0.8680
Validation Loss: 0.2732, Validation Accuracy: 0.8970
Epoch 6/80, Training Loss: 0.3336, Training Accuracy: 0.8759
Epoch 7/80, Training Loss: 0.3167, Training Accuracy: 0.8810
Epoch 8/80, Training Loss: 0.3057, Training Accuracy: 0.8858
Epoch 9/80, Training Loss: 0.2938, Training Accuracy: 0.8897
Epoch 10/80, Training Loss: 0.2841, Training Accuracy: 0.8941
Validation Loss: 0.2382, Validation Accuracy: 0.9144
Epoch 11/80, Training Loss: 0.2746, Training Accuracy: 0.8974
Epoch 12/80, Training Loss: 0.2638, Training Accuracy: 0.9014
Epoch 13/80, Training Loss: 0.2592, Training Accuracy: 0.9027
Epoch 14/80, Training Loss: 0.2541, Training Accuracy: 0.9049
Epoch 15/80, Training Loss: 0.2474,

In [27]:
model.load_state_dict(torch.load('best_model.pth'))

#evaluate on testing set every epoch
model.eval()
running_test_loss = 0.0
correct_test = 0
total_test = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        running_test_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_loss = running_test_loss / len(test_loader.dataset)
test_accuracy = correct_test / total_test

print(f'Testing Loss: {test_loss:.4f}, Testing Accuracy: {test_accuracy:.4f}')

C:\Users\mingyiao\AppData\Local\Temp\ipykernel_18948\1132812897.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Testing Loss: 0.2904, Testing Accuracy: 0.9238
